In [6]:
import websocket
import json
import threading
import time
from dotenv import load_dotenv
import os
load_dotenv()


# Alpaca API credentials
ALPACA_API_KEY = os.getenv('alpaca_key')
ALPACA_API_SECRET = os.getenv('alpaca_secret')

# WebSocket endpoint
WEBSOCKET_URL = "wss://stream.data.alpaca.markets/v2/iex"  # SIP feed (premium)
# For free tier, use "iex" feed instead of "sip":
# WEBSOCKET_URL = "wss://stream.data.alpaca.markets/v2/iex"

def on_open(ws):
    print("WebSocket connection opened.")
    # Authenticate with Alpaca
    auth_message = {
        "action": "auth",
        "key": ALPACA_API_KEY,
        "secret": ALPACA_API_SECRET
    }
    ws.send(json.dumps(auth_message))

    # Subscribe to AAPL stock trades
    subscribe_message = {
        "action": "subscribe",
        "trades": ["AAPL"],  # Subscribe to trades for AAPL
        "quotes": ["AMD","CLDR"],  # You can also subscribe to quotes
        "bars": ["*"]     # Or bars (OHLC)
    }
    ws.send(json.dumps(subscribe_message))

def on_message(ws, message):
    data = json.loads(message)
    print("Message received:", json.dumps(data, indent=2))

def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status_code, close_msg):
    print("WebSocket connection closed.")
    print(f"Close status code: {close_status_code}, message: {close_msg}")

def run_websocket():
    websocket.enableTrace(False)  # Set to True for debugging
    ws = websocket.WebSocketApp(
        WEBSOCKET_URL,
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close
    )
    ws.run_forever()


try:
    websocket_thread = threading.Thread(target=run_websocket)
    websocket_thread.start()

    # Keep the main thread alive
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("WebSocket client stopped.")


WebSocket connection opened.
Message received: [
  {
    "T": "success",
    "msg": "connected"
  }
]
Message received: [
  {
    "T": "error",
    "code": 406,
    "msg": "connection limit exceeded"
  }
]
Message received: [
  {
    "T": "error",
    "code": 401,
    "msg": "not authenticated"
  }
]
WebSocket client stopped.


Error: Connection to remote host was lost.
WebSocket connection closed.
Close status code: None, message: None
